In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from methods import Methods

In [3]:
%matplotlib inline

In [4]:
train = pd.read_csv("Train.csv")

#DATA CLEANING AND EDA

In [6]:
train_copy = train.copy()

In [7]:
train_copy["TOP_PACK"] = Methods.encode_cat_col(train_copy["TOP_PACK"])

In [8]:
train_copy["MRG"] = Methods.encode_cat_col(train_copy["MRG"])

In [9]:
train_copy["REGION"] = Methods.encode_cat_col(train_copy["REGION"])

In [10]:
train_copy["TENURE"] = Methods.encode_cat_col(train_copy["TENURE"])

In [11]:
train_copy["FREQUENCE"].fillna(train_copy["FREQUENCE"].mean(),inplace=True)

In [12]:
train_copy["MONTANT"].fillna(train_copy["MONTANT"].mean(),inplace=True)

In [13]:
train_copy["FREQUENCE_RECH"].fillna(train_copy["FREQUENCE"].mean(),inplace=True)

In [14]:
train_copy["REVENUE"].fillna(train_copy["REVENUE"].mean(),inplace=True)

In [15]:
train_copy["FREQ_TOP_PACK"].fillna(train_copy["FREQ_TOP_PACK"].mean(),inplace=True)

In [16]:
train_copy["ARPU_SEGMENT"].fillna(train_copy["ARPU_SEGMENT"].mean(),inplace=True)

In [17]:
train_copy["DATA_VOLUME"].fillna(train_copy["DATA_VOLUME"].mean(),inplace=True)

In [18]:
train_copy["ON_NET"].fillna(train_copy["ON_NET"].mean(),inplace=True)

In [19]:
train_copy["ORANGE"].fillna(train_copy["ORANGE"].mean(),inplace=True)

In [20]:
train_copy["TIGO"].fillna(train_copy["TIGO"].mean(),inplace=True)

In [21]:
train_copy["ZONE1"].fillna(train_copy["ZONE1"].mean(),inplace=True)

In [22]:
train_copy["ZONE2"].fillna(train_copy["ZONE2"].mean(),inplace=True)

In [23]:
from sklearn.preprocessing import StandardScaler

In [24]:
sc = StandardScaler()

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train,X_test,y_train,y_test = train_test_split(train_copy.drop(["CHURN","user_id"],axis=1),train_copy["CHURN"],test_size=0.2,random_state=42)

In [27]:
sc.fit(X_train)

StandardScaler()

In [28]:
x_trained_scaled = sc.transform(X_train)

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
log = LogisticRegression().fit(x_trained_scaled,y_train)

In [31]:
x_test_scaled = sc.transform(X_test)

In [32]:
predicted = log.predict(x_test_scaled)

In [33]:
from sklearn.metrics import classification_report, confusion_matrix, log_loss

In [34]:
overrall = train_copy.drop(["user_id","CHURN"],axis=1)

In [35]:
overrall = sc.transform(overrall)

In [36]:
res = log.predict(overrall)

In [40]:
test = pd.read_csv("Test.csv")

In [41]:
test_copied = test.copy()

In [43]:
def encode_all_col(df):
    names = df.columns
    fin = dict()
    for name,col in df.iteritems():
        if col.dtype == np.dtype('O'):
            tem = Methods.encode_cat_col(col)
            fin[name] = tem
        else:
            tem = col.fillna(col.mean())
            fin[name] = tem
    return fin

In [44]:
test_final = encode_all_col(test_copied.drop(["user_id"],axis=1))

In [45]:
test_final = pd.DataFrame(test_final)

In [46]:
test_scaled = sc.transform(test_final)

In [47]:
final = log.predict(test_scaled)

In [48]:
sample = pd.DataFrame()
sample["user_id"] = test_copied["user_id"]
sample["CHURN"] = final

In [49]:
sample.to_csv("my_submisiion.csv",index=False)

In [50]:
train_copy["DATA_PER_TENURE"] = train_copy["DATA_VOLUME"] / train_copy["TENURE"]

In [51]:
from sklearn.ensemble import RandomForestClassifier

In [52]:
train_copy.drop(["DATA_PER_TENURE"],axis=1,inplace=True)

In [95]:
rt = RandomForestClassifier(n_estimators=30).fit(x_trained_scaled,y_train)

In [96]:
fake = rt.predict(x_test_scaled)

In [102]:
from sklearn.feature_selection import SelectFromModel
select = SelectFromModel(RandomForestClassifier(n_estimators=100,random_state=42),threshold="median")

In [104]:
select.fit(x_trained_scaled,y_train)

SelectFromModel(estimator=RandomForestClassifier(random_state=42),
                threshold='median')

In [105]:
transformed = select.transform(x_trained_scaled)

In [106]:
trans = select.transform(x_test_scaled)

In [107]:
rt.fit(transformed,y_train)

RandomForestClassifier(n_estimators=30)

In [108]:
pred = rt.predict(trans)

In [109]:
log_loss(pred,y_test)

4.870033408485494

In [110]:
transformed.shape

(320000, 9)

In [130]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10,)).fit(transformed,y_train)

In [131]:
pred = mlp.predict(trans)

In [132]:
log_loss(pred,y_test)

4.713747254387709

In [116]:
final_test = select.transform(test_scaled)

In [133]:
tot = mlp.predict(final_test)

In [134]:
new = pd.DataFrame()
new["user_id"] = test["user_id"]
new["CHURN"] = tot

In [135]:
new.to_csv("new_submit.csv",index=False)